In [1]:
from pydantic import BaseModel
from typing import Optional, Literal
from langchain_core.output_parsers import PydanticOutputParser

from router import Router

In [2]:
# this class define the schema to follow for the output
class State(BaseModel):
    """
    Defines the internal state of the agent.
    """

    input: str

    # here the enumeration of possible next steps
    # important to make optional: at the beginning of the flow it is undefined
    decision: Optional[
        Literal["meetings_info", "places_info", "not_defined", "set_meeting"]
    ] = None
    output: str = ""


# just to see what is generated
# parser = PydanticOutputParser(pydantic_object=State)
# print(parser.get_format_instructions())

In [3]:
router = Router(State)

In [4]:
state = State(input="I want to setup a meeting")

response = router.route(state)
print(response)

input='I want to setup a meeting' decision='set_meeting' output=''


In [5]:
queries = [
    "I want information about restaurants in London",
    "I need the list of the meetings of last week",
    "When I have one hour free next week?",
    "I want to know what I don't know",
    "When in this week I will be free for lunch/dinner?",
    "Give me all the free slot for 2025-02-28",
    "Give me all the available slot for 2025-02-26",
    "Please book a meeting slot for 2025-03-10 11 AM",
    "Set a meeting with Ansh for today 11 AM",
    "Arrange a meeting with Ansh for next tuesday",
]

In [6]:
for query in queries:
    print("")
    print(query)

    response = router.route(State(input=query))
    print("next step is: ", response.decision)


I want information about restaurants in London
next step is:  places_info

I need the list of the meetings of last week
next step is:  meetings_info

When I have one hour free next week?
next step is:  meetings_info

I want to know what I don't know
next step is:  not_defined

When in this week I will be free for lunch/dinner?
next step is:  not_defined

Give me all the free slot for 2025-02-28
next step is:  set_meeting

Give me all the available slot for 2025-02-26
next step is:  meetings_info

Please book a meeting slot for 2025-03-10 11 AM
next step is:  set_meeting

Set a meeting with Ansh for today 11 AM
next step is:  set_meeting

Arrange a meeting with Ansh for next tuesday
next step is:  set_meeting


#### Try with some examples

* Here we show how to pass few examples to improve decision
* In this way I tell alsoe to the model to set only decision and not output

In [7]:
few_shots = [
    {
        "input": "When in this week I will be free for lunch/dinner?",
        "output": {"decision": "meeting_info", "output": ""},
    },
    {
        "input": "Can you arrange a meeting with Alex and Martijn??",
        "output": {"decision": "set_meeting", "output": ""},
    },
]

In [8]:
few_shots

[{'input': 'When in this week I will be free for lunch/dinner?',
  'output': {'decision': 'meeting_info', 'output': ''}},
 {'input': 'Can you arrange a meeting with Alex and Martijn??',
  'output': {'decision': 'set_meeting', 'output': ''}}]

In [9]:
router = Router(model=State, few_shot_examples=few_shots)

In [10]:
for query in queries:
    print("")
    print(query)

    response = router.route(State(input=query))
    print("next step is: ", response.decision)


I want information about restaurants in London
next step is:  places_info

I need the list of the meetings of last week
next step is:  meetings_info

When I have one hour free next week?
next step is:  meetings_info

I want to know what I don't know
next step is:  not_defined

When in this week I will be free for lunch/dinner?
next step is:  meetings_info

Give me all the free slot for 2025-02-28
next step is:  meetings_info

Give me all the available slot for 2025-02-26
next step is:  meetings_info

Please book a meeting slot for 2025-03-10 11 AM
next step is:  set_meeting

Set a meeting with Ansh for today 11 AM
next step is:  set_meeting

Arrange a meeting with Ansh for next tuesday
next step is:  set_meeting
